In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import cv2
import mediapipe as mp
import os
import subprocess
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Utility Function

In [4]:
def count_file(directory):
    file_count_per_class = {}
    total_files = 0

    for root, dirs, files in os.walk(directory):
        class_name = root.split(os.sep)[-1]
        count = sum(1 for file in files if file.endswith(".png") or file.endswith(".jpg"))
        
        if count > 0:
            file_count_per_class[class_name] = count
            total_files += count

    for class_name, count in file_count_per_class.items():
        print(f"{class_name}: {count}")

    print(f"Total Dataset: {total_files}")
    
count_file('whisnu-dataset-resize')

bagaimana: 377
besok: 388
hari ini: 326
jawab: 373
kantor: 370
kemarin: 376
kerja: 442
lusa: 318
malam: 354
nanti: 347
pagi: 376
sekarang: 307
siang: 348
sore: 309
Total Dataset: 5011


# Generate Annotated Raw

In [ ]:
def get_valid_landmarks(hand_landmarks, image_width, image_height):
    valid_landmarks = []
    for landmark in hand_landmarks.landmark:
        # Convert landmark coordinates to pixel values
        x = int(landmark.x * image_width)
        y = int(landmark.y * image_height)
        
        # Check if the landmark is within the image bounds
        if 0 <= x < image_width and 0 <= y < image_height:
            valid_landmarks.append((x, y))
    
    return valid_landmarks


def annotate_hand_images(input_directory, output_directory):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_hands = mp.solutions.hands

    hand_notfound_images = []
    incomplete_landmark_hand = []

    os.makedirs(output_directory, exist_ok=True)

    for root, dirs, files in os.walk(input_directory):
        for file in files:
            if file.lower().endswith(".jpg"):
                input_file = os.path.join(root, file)
                output_subdir = os.path.join(output_directory, os.path.relpath(root, input_directory))
                os.makedirs(output_subdir, exist_ok=True)

                with mp_hands.Hands(
                    static_image_mode=True,
                    max_num_hands=2,
                    min_detection_confidence=0.5,
                ) as hands:
                    filename = os.path.splitext(file)[0]
                    print(f"Processing: {filename}")
                    image = cv2.flip(cv2.imread(input_file), 1)
                    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                    if not results.multi_hand_landmarks:
                        hand_notfound_images.append(input_file)
                        continue

                    image_height, image_width, _ = image.shape
                    annotated_image = image.copy()
                    output_file = os.path.join(output_subdir, f"annotated_{filename}.jpg")

                    for hand_landmarks in results.multi_hand_landmarks:
                        valid_landmarks = get_valid_landmarks(hand_landmarks, image_width, image_height)
                        if len(valid_landmarks) <  21:
                            incomplete_landmark_hand.append(output_file)
                        
                        mp_drawing.draw_landmarks(
                            annotated_image,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS,
                            mp_drawing_styles.get_default_hand_landmarks_style(),
                            mp_drawing_styles.get_default_hand_connections_style())

                    cv2.imwrite(output_file, cv2.flip(annotated_image, 1))
    
    return hand_notfound_images, incomplete_landmark_hand

In [19]:
# Example usage
input_directory = "whisnu-dataset-resize"
output_directory = "whisnu-annotated-raw_dataset"
hand_notfound_images, incomplete_landmark_hand = annotate_hand_images(input_directory, output_directory)
# Optionally, print or save lists of images where hands were not found or landmarks were incomplete
print("Images with no hands detected:", len(hand_notfound_images))
print("Images with incomplete hand landmarks:", len(incomplete_landmark_hand))

Processing: 0_bagaimana_1


c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing: 0_bagaimana_2
Processing: 100_bagaimana_1
Processing: 100_bagaimana_2
Processing: 101_bagaimana_1
Processing: 101_bagaimana_2
Processing: 102_bagaimana_1
Processing: 102_bagaimana_2
Processing: 103_bagaimana_1
Processing: 103_bagaimana_2
Processing: 104_bagaimana_1
Processing: 104_bagaimana_2
Processing: 105_bagaimana_1
Processing: 105_bagaimana_2
Processing: 106_bagaimana_1
Processing: 106_bagaimana_2
Processing: 107_bagaimana_1
Processing: 107_bagaimana_2
Processing: 108_bagaimana_1
Processing: 108_bagaimana_2
Processing: 109_bagaimana_1
Processing: 109_bagaimana_2
Processing: 10_bagaimana_1
Processing: 10_bagaimana_2
Processing: 110_bagaimana_1
Processing: 110_bagaimana_2
Processing: 111_bagaimana_1
Processing: 111_bagaimana_2
Processing: 112_bagaimana_1
Processing: 112_bagaimana_2
Processing: 113_bagaimana_1
Processing: 113_bagaimana_2
Processing: 114_bagaimana_1


KeyboardInterrupt: 

# Filter Image in Hand Not Found and Incomplete Landmark

In [15]:
def get_ori_file(file_path):
    file_name = file_path.split('\\')[-1].replace('annotated_', '')
    class_name = file_name.split('_')[1]
    ori_path_file = os.path.join(f"whisnu-dataset-resize\\{class_name}", file_name)
    return ori_path_file

print(hand_notfound_images[:5])
print(incomplete_landmark_hand[:5])

raw_incomplete_landmark_hand = [get_ori_file(file_path) for file_path in incomplete_landmark_hand]

bad_image = hand_notfound_images + raw_incomplete_landmark_hand
print(bad_image[:2])
print(bad_image[-2:])

['whisnu-dataset-resize\\besok\\100_besok_1.jpg', 'whisnu-dataset-resize\\besok\\100_besok_2.jpg', 'whisnu-dataset-resize\\hari ini\\165_hari_ini1.jpg', 'whisnu-dataset-resize\\hari ini\\165_hari_ini2.jpg', 'whisnu-dataset-resize\\jawab\\112_jawab_1.jpg']
['whisnu-annotated-raw_dataset\\bagaimana\\annotated_202_bagaimana_1.jpg', 'whisnu-annotated-raw_dataset\\bagaimana\\annotated_202_bagaimana_2.jpg', 'whisnu-annotated-raw_dataset\\bagaimana\\annotated_203_bagaimana_1.jpg', 'whisnu-annotated-raw_dataset\\bagaimana\\annotated_203_bagaimana_2.jpg', 'whisnu-annotated-raw_dataset\\bagaimana\\annotated_204_bagaimana_1.jpg']
['whisnu-dataset-resize\\besok\\100_besok_1.jpg', 'whisnu-dataset-resize\\besok\\100_besok_2.jpg']
['whisnu-dataset-resize\\sore\\96_sore_1.jpg', 'whisnu-dataset-resize\\sore\\96_sore_2.jpg']


In [16]:
def hapus_file(list_path):
    for path in list_path:
        try:
            if os.path.isfile(path):
                os.remove(path)
                print(f"File {path} berhasil dihapus.")
            else:
                print(f"File {path} tidak ditemukan atau bukan file.")
        except Exception as e:
            print(f"Terjadi kesalahan saat menghapus file {path}: {e}")

hapus_file(bad_image)

File whisnu-dataset-resize\besok\100_besok_1.jpg berhasil dihapus.
File whisnu-dataset-resize\besok\100_besok_2.jpg berhasil dihapus.
File whisnu-dataset-resize\hari ini\165_hari_ini1.jpg berhasil dihapus.
File whisnu-dataset-resize\hari ini\165_hari_ini2.jpg berhasil dihapus.
File whisnu-dataset-resize\jawab\112_jawab_1.jpg berhasil dihapus.
File whisnu-dataset-resize\jawab\113_jawab_1.jpg berhasil dihapus.
File whisnu-dataset-resize\jawab\213_jawab_1.jpg berhasil dihapus.
File whisnu-dataset-resize\jawab\214_jawab_1.jpg berhasil dihapus.
File whisnu-dataset-resize\jawab\237_jawab_1.jpg berhasil dihapus.
File whisnu-dataset-resize\jawab\247_jawab_1.jpg berhasil dihapus.
File whisnu-dataset-resize\jawab\248_jawab_1.jpg berhasil dihapus.
File whisnu-dataset-resize\jawab\249_jawab_1.jpg berhasil dihapus.
File whisnu-dataset-resize\jawab\251_jawab_1.jpg berhasil dihapus.
File whisnu-dataset-resize\jawab\261_jawab_1.jpg berhasil dihapus.
File whisnu-dataset-resize\jawab\262_jawab_1.jpg ber

# Delete Manual Image with Landmark not Precise

how it works? delete data from annotated raw with imprecise landmarks => run code in below

In [2]:
def list_images(directory):
    return [
        os.path.join(root, f)
        for root, _, files in os.walk(directory)
        for f in files
        if f.lower().endswith((".jpg", ".png"))
    ]
    
def delete_dir2_based_dir1(dir1, dir2):
    annotated_files = set(list_images(dir1))
    augmented_files = set(list_images(dir2))

    annotated_files = [os.path.basename(file) for file in annotated_files]
    count = 0
    for file in augmented_files:
        # check if the file is not in annotated_files
        # first add prefix 'annotated_' to the file name
        prefixed_file = "annotated_" + os.path.basename(file)

        if prefixed_file not in annotated_files:
            os.remove(file)
            print(f"File {file} berhasil dihapus.")
            count += 1
    
    print(f"Total file yang dihapus: {count}")

In [3]:
ANNOTATED_DIR = './whisnu-annotated-raw_dataset'
DATA_CLEAN_DIR = './whisnu-dataset-resize'

delete_dir2_based_dir1(ANNOTATED_DIR, DATA_CLEAN_DIR)

File ./whisnu-dataset-resize\hari ini\20_hari_ini2.jpg berhasil dihapus.
File ./whisnu-dataset-resize\bagaimana\33_bagaimana_1.jpg berhasil dihapus.
File ./whisnu-dataset-resize\kantor\277_kantor_1.jpg berhasil dihapus.
File ./whisnu-dataset-resize\sore\84_sore_2.jpg berhasil dihapus.
File ./whisnu-dataset-resize\bagaimana\31_bagaimana_1.jpg berhasil dihapus.
File ./whisnu-dataset-resize\sore\44_sore_2.jpg berhasil dihapus.
File ./whisnu-dataset-resize\kantor\299_kantor_1.jpg berhasil dihapus.
File ./whisnu-dataset-resize\kantor\378_kantor_1.jpg berhasil dihapus.
File ./whisnu-dataset-resize\malam\87_malam_1.jpg berhasil dihapus.
File ./whisnu-dataset-resize\kantor\447_kantor_1.jpg berhasil dihapus.
File ./whisnu-dataset-resize\kantor\54_kantor_1.jpg berhasil dihapus.
File ./whisnu-dataset-resize\kantor\218_kantor_1.jpg berhasil dihapus.
File ./whisnu-dataset-resize\bagaimana\44_bagaimana_2.jpg berhasil dihapus.
File ./whisnu-dataset-resize\kantor\110_kantor_1.jpg berhasil dihapus.
Fil

NameError: name 'count_file' is not defined

In [5]:
print('\nImage Left')
count_file('whisnu-dataset-resize')


Image Left
bagaimana: 377
besok: 388
hari ini: 326
jawab: 373
kantor: 370
kemarin: 376
kerja: 442
lusa: 318
malam: 354
nanti: 347
pagi: 376
sekarang: 307
siang: 348
sore: 309
Total Dataset: 5011


# Extraksi Fitur

In [6]:
def extract_hand_features(input_dir, output_filename):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5
    )

    data = []

    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image_rgb)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    landmarks = []
                    for landmark in hand_landmarks.landmark:
                        landmarks.append(landmark.x)
                        landmarks.append(landmark.y)
                        landmarks.append(landmark.z)
                    data.append([filename] + landmarks)

    hands.close()
    

    columns = ["filename"]
    for i in range(21):  # 21 landmarks
        columns += [f"x_{i}", f"y_{i}", f"z_{i}"]

    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_filename, index=False)


out_dir = "./whisnu-extraction_feature-dataset/raw"
os.makedirs(out_dir, exist_ok=True)
DATA_DIR = './whisnu-dataset-resize'

for files in os.listdir(DATA_DIR):
    input_dir = os.path.join(DATA_DIR, files)
    output_filename = os.path.join(out_dir, f"{files}.csv")
    extract_hand_features(input_dir, output_filename)

c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


NotADirectoryError: [WinError 267] The directory name is invalid: './whisnu-dataset-resize\\whisnu.zip'

# Now Let's Go Augmented

Goal
1. Augmented 3-5k (let's try 5k each class)
2. Generate Annotated